In [1]:
debug = False

from picamera import PiCamera
from picamera.array import PiRGBArray, PiYUVArray
from scipy.signal import filtfilt, butter, find_peaks
from pwm import PWM
import sys
from time import sleep
import threading
import ipywidgets as widgets


In [2]:
minSpeed =  1000000 # 0 speed
zeroRot  =  1000000
maxRot   =   500000
rot = zeroRot


pwm0 = PWM(0)  # motor
# pwm0.export()
pwm0.period = 20000000
pwm0.duty_cycle = minSpeed

pwm1 = PWM(1)  # servo
# pwm1.export() 
pwm1.period = 20000000
pwm1.duty_cycle = zeroRot 

pwm0.enable = True
pwm1.enable = True

if not debug:
    print("waiting for motor")
    sleep(3)
    print("motor ready")

waiting for motor
motor ready


In [3]:
roi_perc = 0.5
peaks_threshold = 0.9
threshold_deviation = 0.15
kp = 2500
speed = 1150000
terminate = False
# if debug:
#     speed = minSpeed

def set_r_s(r):
    global roi_perc
    roi_perc = r["new"]

def set_p_s(p):
    global peaks_threshold
    peaks_threshold = p["new"]
    
def set_t_s(t):
    global threshold_deviation
    threshold_deviation = t["new"]  

def set_k_s(k):
    global kp
    kp = k["new"]      

def set_s_s(s):
    global speed
    speed = s["new"] 

def set_o_s(o):
    global terminate
    terminate = o["new"] 


r_s = widgets.FloatSlider(description="ROI line", value=0.8, min=0.0, max=1.0)
p_s = widgets.FloatSlider(description="Peak threshold", value=0.9, min=0.0, max=1.0)
t_s = widgets.FloatSlider(description="Threshold deviation", value=0.15, min=0.0, max=1.0)
k_s = widgets.IntSlider(description="Kp", value=2500, min=1000, max=4000, step = 50)
s_s = widgets.IntSlider(description="Speed", value=1200000, min=1000000, max=2000000, step=100000)
o_s = widgets.Checkbox(description="Terminate", value=False, disabled=False)

r_s.observe(set_r_s, "value")
p_s.observe(set_p_s, "value")
t_s.observe(set_t_s, "value")
k_s.observe(set_k_s, "value")
s_s.observe(set_s_s, "value")
o_s.observe(set_o_s, "value")
display(r_s, p_s, t_s, k_s, s_s, o_s)

FloatSlider(value=0.8, description='ROI line', max=1.0)

FloatSlider(value=0.9, description='Peak threshold', max=1.0)

FloatSlider(value=0.15, description='Threshold deviation', max=1.0)

IntSlider(value=2500, description='Kp', max=4000, min=1000, step=50)

IntSlider(value=1200000, description='Speed', max=2000000, min=1000000, step=100000)

Checkbox(value=False, description='Terminate')

In [4]:
# kp should be around 2500.
# ki
# get the error. Error is peak position - middle.
# servo angle is 1.5 million + kp*error
def t_func():
    global rot
    global roi_perc
    global peaks_threshold
    global threshold_deviation
    global kp
    global terminate
    global speed
    
    rawCapture = PiYUVArray(camera)
    stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)
    b, a = butter(3, 0.007)
    try:
        for frame in stream:
            I = frame.array[:, :, 0]
            rawCapture.truncate(0)
            
            mid_horizontal = int(I.shape[0]*0.4)
            mid_vertical = int(I.shape[1]/2)
            mid_horizontal1 =int(I.shape[0]*(0.5))
            mid_horizontal2 =int(I.shape[0]*(0.9))
            L = I[mid_horizontal, :]
            L1=I[mid_horizontal1, :]
            L2=I[mid_horizontal2, :]
            Lf = filtfilt(b, a, L)
            Lf1= filtfilt(b, a, L1)
            Lf2=filtfilt(b,a,L2)
            #print(Lf)
            #print(Lf1)
            #print(Lf2)

            # normalize LF
            #max_Lf = max(max(Lf),max(Lf1),max(Lf2))
            #min_Lf = min(min(Lf),min(Lf1),min(Lf2))
            max_Lf=max(Lf)
            min_Lf=min(Lf)
            max_Lf1=max(Lf1)
            min_Lf1=min(Lf1)
            max_Lf2=max(Lf2)
            min_Lf2=min(Lf2)
            Lf = [(val - min_Lf)/(max_Lf - min_Lf) for val in Lf]
            Lf1 = [(val - min_Lf)/(max_Lf1 - min_Lf1) for val in Lf1]
            Lf2 = [(val - min_Lf)/(max_Lf2 - min_Lf2) for val in Lf2]
            #print(Lf)
            #print(Lf1)
            #print(Lf2)
            p = find_peaks(Lf, peaks_threshold)
            p1 = find_peaks(Lf1, peaks_threshold)
            p2 = find_peaks(Lf2, peaks_threshold)
            #print(p)
            #print(p1)
            #print(p2)
            closest_peak_at = -1
            distance_peak = 1000000
            closest_peak_at1 = -1
            distance_peak1 = 1000000
            closest_peak_at2 = -1
            distance_peak2 = 1000000
            for peak in p[0]:
                if abs(mid_vertical - peak) < distance_peak:
                    closest_peak_at = peak
                    distance_peak = abs(mid_vertical - peak)
            for peak in p1[0]:
                if abs(mid_vertical1 - peak) < distance_peak1:
                    closest_peak_at1 = peak
                    distance_peak1 = abs(mid_vertical - peak)
                                
            for peak in p2[0]:
                if abs(mid_vertical - peak) < distance_peak2:
                    closest_peak_at2 = peak
                    distance_peak2 = abs(mid_vertical - peak)
            
            deviation=((mid_vertical - closest_peak_at)*1.0)/mid_vertical
            print(deviation)
            deviation1=((mid_vertical - closest_peak_at1)*1.0)/mid_vertical
            print(deviation)
            deviation2=((mid_vertical - closest_peak_at2)*1.0)/mid_vertical
            print(deviation)
#             deviation = ((mid_vertical - closest_peak_at)*1.0)/mid_vertical

#             if deviation < -1*threshold_deviation:
#                 move = "right"
#             elif -1*threshold_deviation <= deviation <= threshold_deviation:
#                 move = "straight"
#             else:
#                 move = "left"
            error = mid_horizontal - closest_peak_at
            error1 = mid_horizontal1 - closest_peak_at1
            error2 = mid_horizontal2 - closest_peak_at2
            if(
                (-1*threshold_deviation <= deviation <= threshold_deviation) and 
                (-1*threshold_deviation <= deviation1 <= threshold_deviation) and
                (-1*threshold_deviation <= deviation2 <= threshold_deviation)
            ):
            
            
                new_angle = zeroRot + kp*error
                new_angle = max(zeroRot - maxRot, new_angle)
                new_angle = min(zeroRot + maxRot, new_angle)
                rot = new_angle
#             if move == 'left':
#                 rot = min(zeroRot + maxRot, rot+0.10*rot)
#             elif move == 'right':
#                 rot = max(zeroRot -maxRot, rot-0.10*rot)
#             else:
#                 rot = zeroRot

#             print(speed, roi_perc, peaks_threshold, threshold_deviation)
                pwm0.duty_cycle = speed*1.10
                pwm1.duty_cycle = int(rot)
            else:
                new_angle = zeroRot + kp*error
                new_angle = max(zeroRot - maxRot, new_angle)
                new_angle = min(zeroRot + maxRot, new_angle)
                rot = new_angle
                pwm0.duty_cycle = speed
                pwm1.duty_cycle = int(rot)
            if terminate:
                print("YAY! TERMINATING !!!!")
                pwm0.duty_cycle = minSpeed
                pwm1.duty_cycle = zeroRot
                stream.close()
                rawCapture.close()
                camera.close()
                break
    except:
        pwm0.duty_cycle = minSpeed
        pwm1.duty_cycle = zeroRot
        stream.close()
        rawCapture.close()
        camera.close()

In [5]:
camera = PiCamera()
# camera.sensor_mode = 7
# # camera.resolution = res
# time.sleep(2)
# camera.framerate = 120
# camera.shutter_speed = camera.exposure_speed
# camera.exposure_mode = 'off'
# g = camera.awb_gains
# camera.awb_mode = 'off'
# camera.awb_gains = g
t = threading.Thread(target = t_func)
t.start()

/home/pi/.virtualenvs/raceon/lib/python3.5/site-packages/picamera/encoders.py:544: PiCameraResolutionRounded: frame size rounded up from 720x480 to 736x480
  width, height, fwidth, fheight)))


[ 32.49918466  32.457869    32.41521663  32.37122515  32.32589291
  32.27921901  32.23120328  32.18184638  32.13114973  32.07911558
  32.025747    31.97104793  31.91502316  31.85767837  31.79902013
  31.73905594  31.67779422  31.61524435  31.55141666  31.48632247
  31.41997409  31.35238484  31.28356906  31.21354214  31.14232051
  31.06992167  30.99636421  30.9216678   30.84585323  30.7689424
  30.69095834  30.61192524  30.53186843  30.45081442  30.36879089
  30.28582673  30.20195202  30.11719806  30.03159736  29.9451837
  29.85799207  29.77005873  29.6814212   29.59211829  29.50219007
  29.41167791  29.32062448  29.22907374  29.13707098  29.0446628
  28.95189712  28.85882321  28.76549164  28.67195434  28.57826458
  28.48447695  28.39064742  28.29683327  28.20309314  28.10948702
  28.01607622  27.92292342  27.83009262  27.73764916  27.6456597
  27.55419224  27.4633161   27.3731019   27.28362157  27.19494836
  27.10715677  27.02032263  26.93452301  26.84983626  26.76634197
  26.68412098 

PiCameraValueError: Incorrect buffer length for resolution 720x480